In [1]:
from AppInspector.SharingInstance import SharingInstance, obj
import hashlib
import json
import os
from Learner import Learner
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from utils import Utilities
from sklearn import svm
from sklearn.linear_model import LogisticRegression
import pandas as pd


In [2]:
logger = Utilities.set_logger('InstanceHandler')
logger.setLevel(level=20)

Formatting the training instances, the data of each instance is a string.

In [3]:
def docs(instances):
    docs = []
    labels = []
    for instance in instances:
        doc = []
        for string in instance.ui_doc:
            doc.append(' '.join(Learner.str2words(str(string))))
        docs.append(' '.join(doc))
        labels.append(int(instance.label))
    return docs, np.array(labels)


Trainig and cross-validation

In [4]:
def handle(root_dir):
    # instances_dir_name = hashlib.md5(root_dir.encode('utf-8')).hexdigest()
    instances_dir_path = os.path.join('data', os.path.basename(root_dir))
    pos_dir = os.path.join(root_dir, '1')
    pos_out_dir = os.path.join(instances_dir_path, '1')
    neg_dir = os.path.join(root_dir, '0')
    neg_out_dir = os.path.join(instances_dir_path, '0')
    if not os.path.exists(instances_dir_path):
        os.makedirs(pos_out_dir)
        instances = SharingInstance.instances(pos_dir)
        for instance in instances:
            with open(os.path.join(pos_out_dir, instance.id + '.json'), 'w', encoding="utf8") as outfile:
                outfile.write(instance.json())

        os.makedirs(neg_out_dir)
        neg_instances = SharingInstance.instances(neg_dir)
        logger.info('neg: ' + str(len(neg_instances)))
        for instance in neg_instances:
            with open(os.path.join(neg_out_dir, instance.id + '.json'), 'w', encoding="utf8") as outfile:
                outfile.write(instance.json())
        instances += neg_instances
    else:
        instances = []
        instances_dict = []
        for dir_path in [pos_out_dir, neg_out_dir]:
            for root, dirs, files in os.walk(dir_path):
                for file_name in files:
                    if file_name.endswith('.json'):
                        with open(os.path.join(root, file_name), 'r', encoding="utf8") as my_file:
                            instance = json.load(my_file)
                            instances_dict.append(instance)
                            instance = obj(instance)
                            logger.debug(instance.dir)
                            instances.append(instance)
        with open(os.path.join(instances_dir_path, 'instances.json'), 'w', encoding="utf8") as outfile:
            json.dump(instances_dict, outfile)
            # pd.Series(instances).to_json(outfile, orient='values')
    docs, y = docs(instances)
    train_data, voc, vec = Learner.gen_X_matrix(docs)
    logger.info('neg: ' + str(len(np.where(y == 0)[0])))
    folds = Learner.n_folds(train_data, y, fold=10)  #[Fold(f) for f in Learner.n_folds(train_data, y, fold=10)]
    """
    clf = DecisionTreeClassifier(class_weight='balanced')
    res = Learner.cross_validation(clf, folds)
    for fold in res['fold']:
        for item in fold['fp_item']:
            instance = instances[item]
            InstanceHandler.logger.info("FP:" + str(item) + str(instance.ui_doc) + "," + str(instance.dir))
        for item in fold['fn_item']:
            instance = instances[item]
            InstanceHandler.logger.info("FN:" + str(item) + str(instance.ui_doc) + "," + str(instance.dir))
    
    clf = MultinomialNB()
    Learner.cross_validation(clf, folds)
    clf = RandomForestClassifier(class_weight='balanced')
    Learner.cross_validation(clf, folds)
    clf = svm.SVC(kernel='linear', class_weight='balanced', probability=True)
    Learner.cross_validation(clf, folds)
    clf = LogisticRegression(class_weight='balanced')
    Learner.cross_validation(clf, folds)
    """
    clfs = [svm.SVC(kernel='linear', class_weight='balanced', probability=True),
            RandomForestClassifier(class_weight='balanced'),
            LogisticRegression(class_weight='balanced')]
    res = Learner.voting(clfs, train_data, y, folds)
    with open(os.path.join(instances_dir_path, 'folds.json'), 'w') as json_file:
        for fold in folds:
            fold['train_index'] = fold['train_index'].tolist()
            fold['test_index'] = fold['test_index'].tolist()
        # pd.Series(folds).to_json(json_file, orient='values')
        logger.info(len(folds))
        json.dump(folds, json_file)
    """
    with open(os.path.join(instances_dir_path, 'voting_res.json'), 'w') as json_file:
        pd.Series(res).to_json(json_file, orient='split')
        # json.dump(res, json_file)
    with open(os.path.join(instances_dir_path, 'voting_predicted_neg.json'), 'w') as json_file:
        json.dump(predicted_neg_instances, json_file)
    """


We can now 

In [5]:
root_dir = '/mnt/H_DRIVE/FlowIntent/Location/'
handle(root_dir)

/mnt/H_DRIVE/FlowIntent/Location/1
google
6
1-4
5
/mnt/H_DRIVE/FlowIntent/Location/1
edu
tools
fin
news_not_done
person
health
shop
enter
busin
sport
commu
special
social
map
weather
med
/mnt/H_DRIVE/FlowIntent/Location/1
com.runar.issdetector
/mnt/H_DRIVE/FlowIntent/Location/1/google/edu/com.runar.issdetector
['Next:', '1h 53m 22s', 'Davis', 'Saturday, 25 Jul', 'ISS', '02:10:25', 'Mag 1.7', '02:11:12', '13°', 'ISS', '05:22:30', 'Mag 1.3', '05:26:20', '15°', 'Iridium 97', '23:05:22', 'Mag -1.1', '10°', 'Sunday, 26 Jul', 'ISS', '04:29:54', 'Mag 1.6', '04:31:54', '11°', 'Monday, 27 Jul', 'ISS', '02:00:03']
com.lavadip.skeye
/mnt/H_DRIVE/FlowIntent/Location/1/google/edu/com.lavadip.skeye
['Push Notifications', '7NEWS Denver can occasionally send you push notification for breaking news alerts. Would you like to receive these notifications? You can also change your selection later from the Settings page in the app.', 'No', 'Yes']
com.SharpSchool.SharpSchoolMobile
/mnt/H_DRIVE/FlowIntent/Loc

['CATEGORIES', 'weather', 'FESTIVALS', 'WIN A TABLET', 'news', 'OTHER FESTIVALS', 'ENTERTAINMENT CALENDAR', 'VIDEOS AND PHOTOS', 'NEWS', 'WEATHER AUTHORITY', 'FOX 8 ON FACEBOOK', 'FOX 8 ON TWITTER', 'SAVED STORIES', 'SAVED PHOTOS', 'SAVED VIDEOS', 'FESTIVALS', 'Mar 24, 2015 12:10 PM', '7/4 Go 4th on the River', 'Mar 24, 2015 12:22 PM', '7/9-7/12 Running of the Bulls', 'Jan 13, 2015 8:42 AM', '7/15-7/19 Tales of the Cocktail', 'Jan 13, 2015 8:42 AM']
jpost.webActivity0629-19-09-54
/mnt/H_DRIVE/FlowIntent/Location/1/google/news_not_done/jpost.webActivity0629-19-09-54
['CATEGORIES', 'weather', 'FESTIVALS', 'WIN A TABLET', 'news', 'OTHER FESTIVALS', 'ENTERTAINMENT CALENDAR', 'VIDEOS AND PHOTOS', 'NEWS', 'WEATHER AUTHORITY', 'FOX 8 ON FACEBOOK', 'FOX 8 ON TWITTER', 'SAVED STORIES', 'SAVED PHOTOS', 'SAVED VIDEOS', 'FESTIVALS', 'Mar 24, 2015 12:10 PM', '7/4 Go 4th on the River', 'Mar 24, 2015 12:22 PM', '7/9-7/12 Running of the Bulls', 'Jan 13, 2015 8:42 AM', '7/15-7/19 Tales of the Cocktail'

['CATEGORIES', 'weather', 'FESTIVALS', 'WIN A TABLET', 'news', 'OTHER FESTIVALS', 'ENTERTAINMENT CALENDAR', 'VIDEOS AND PHOTOS', 'NEWS', 'WEATHER AUTHORITY', 'FOX 8 ON FACEBOOK', 'FOX 8 ON TWITTER', 'SAVED STORIES', 'SAVED PHOTOS', 'SAVED VIDEOS', 'FESTIVALS', 'Mar 24, 2015 12:10 PM', '7/4 Go 4th on the River', 'Mar 24, 2015 12:22 PM', '7/9-7/12 Running of the Bulls', 'Jan 13, 2015 8:42 AM', '7/15-7/19 Tales of the Cocktail', 'Jan 13, 2015 8:42 AM']
com.nbcuni.nbcots.nbcwashington.android0629-13-58-00
/mnt/H_DRIVE/FlowIntent/Location/1/google/news_not_done/com.nbcuni.nbcots.nbcwashington.android0629-13-58-00
['Push Notifications', '7NEWS Denver can occasionally send you push notification for breaking news alerts. Would you like to receive these notifications? You can also change your selection later from the Settings page in the app.', 'No', 'Yes']
com.news13.news13plus0629-14-59-31
/mnt/H_DRIVE/FlowIntent/Location/1/google/news_not_done/com.news13.news13plus0629-14-59-31
['Push Notifi

['KABB FOX29', 'News', 'Weather', 'Watch Live Here', 'Live Events', 'Sports', 'Daytime @ Nine']
com.raycom.wsfa0629-21-50-28
/mnt/H_DRIVE/FlowIntent/Location/1/google/news_not_done/com.raycom.wsfa0629-21-50-28
['CATEGORIES', 'weather', 'FESTIVALS', 'WIN A TABLET', 'news', 'OTHER FESTIVALS', 'ENTERTAINMENT CALENDAR', 'VIDEOS AND PHOTOS', 'NEWS', 'WEATHER AUTHORITY', 'FOX 8 ON FACEBOOK', 'FOX 8 ON TWITTER', 'SAVED STORIES', 'SAVED PHOTOS', 'SAVED VIDEOS', 'FESTIVALS', 'Mar 24, 2015 12:10 PM', '7/4 Go 4th on the River', 'Mar 24, 2015 12:22 PM', '7/9-7/12 Running of the Bulls', 'Jan 13, 2015 8:42 AM', '7/15-7/19 Tales of the Cocktail', 'Jan 13, 2015 8:42 AM']
com.ap.advnola0629-18-36-23
/mnt/H_DRIVE/FlowIntent/Location/1/google/news_not_done/com.ap.advnola0629-18-36-23
['CATEGORIES', 'weather', 'FESTIVALS', 'WIN A TABLET', 'news', 'OTHER FESTIVALS', 'ENTERTAINMENT CALENDAR', 'VIDEOS AND PHOTOS', 'NEWS', 'WEATHER AUTHORITY', 'FOX 8 ON FACEBOOK', 'FOX 8 ON TWITTER', 'SAVED STORIES', 'SAVED P

['CATEGORIES', 'weather', 'FESTIVALS', 'WIN A TABLET', 'news', 'OTHER FESTIVALS', 'ENTERTAINMENT CALENDAR', 'VIDEOS AND PHOTOS', 'NEWS', 'WEATHER AUTHORITY', 'FOX 8 ON FACEBOOK', 'FOX 8 ON TWITTER', 'SAVED STORIES', 'SAVED PHOTOS', 'SAVED VIDEOS', 'FESTIVALS', 'Mar 24, 2015 12:10 PM', '7/4 Go 4th on the River', 'Mar 24, 2015 12:22 PM', '7/9-7/12 Running of the Bulls', 'Jan 13, 2015 8:42 AM', '7/15-7/19 Tales of the Cocktail', 'Jan 13, 2015 8:42 AM']
com.raycom.wfie0629-03-48-52
/mnt/H_DRIVE/FlowIntent/Location/1/google/news_not_done/com.raycom.wfie0629-03-48-52
['CATEGORIES', 'weather', 'FESTIVALS', 'WIN A TABLET', 'news', 'OTHER FESTIVALS', 'ENTERTAINMENT CALENDAR', 'VIDEOS AND PHOTOS', 'NEWS', 'WEATHER AUTHORITY', 'FOX 8 ON FACEBOOK', 'FOX 8 ON TWITTER', 'SAVED STORIES', 'SAVED PHOTOS', 'SAVED VIDEOS', 'FESTIVALS', 'Mar 24, 2015 12:10 PM', '7/4 Go 4th on the River', 'Mar 24, 2015 12:22 PM', '7/9-7/12 Running of the Bulls', 'Jan 13, 2015 8:42 AM', '7/15-7/19 Tales of the Cocktail', 'J

...


['Keyboard…']
['Contact Us', '©2016 mymixtapez.com', 'App version 6.0.65', 'Terms and Conditions.', 'Privacy Policy', 'Send Feedback/Tech Support', 'If you are contacting us about uploading a mixtape or any technical issue on the app.', 'Sales and Promotion', 'Interested in having your mixtape featured on the front?\nGet in touch with our sales team.', '.................................................................................................................', 'Digital Millennium Copyright Act.', 'If you are legal representative, contact us.']
['Dude', '450', 'Favorites', '3540', 'Followers', '239', 'Following', 'Edit Profile', 'Guilherme Dourado', "When I talk about money all you see is the struggle/ When I tell you I'm livin' large, you tell me it's trouble.", 'Recent Played Artists']
['E-mail / Username', 'Log in', 'Sign in with Google', "Don't have an account?", 'Create Account', 'Forgot your password?', 'Click here', 'Skip', 'registering for now']
[]
[]
['Dude', '450', 'Fa


['Recently we have released version completely new application, this version has improved performance and integrating all existing languages, please visit http://mangaworldapp.com to download the latest version of the app. With the new version, when you sign in with your Google Account in the app, "Remove Ad" will still be restored.', 'No', 'Yes']
['Recently we have released version completely new application, this version has improved performance and integrating all existing languages, please visit http://mangaworldapp.com to download the latest version of the app. With the new version, when you sign in with your Google Account in the app, "Remove Ad" will still be restored.', 'No', 'Yes']
['Recently we have released version completely new application, this version has improved performance and integrating all existing languages, please visit http://mangaworldapp.com to download the latest version of the app. With the new version, when you sign in with your Google Account in the app, 

2018-02-16 20:25:35,197 - InstanceHandler - INFO - neg: 1025



com.mangaworld2.manga_russian2
/mnt/H_DRIVE/FlowIntent/Location/0/Play_win8/com.mangaworld2.manga_russian2
['Недавно мы выпустили версию совершенно нового приложения, эта версия улучшила производительность и интегрировала все существующие языки, пожалуйста, посетите http://mangaworldapp.com для загрузки последней версии приложения. С новой версией при входе в аккаунт с учетной записью Google «Удалить объявление» все равно будет восстановлено.', 'нет', 'да']
['Недавно мы выпустили версию совершенно нового приложения, эта версия улучшила производительность и интегрировала все существующие языки, пожалуйста, посетите http://mangaworldapp.com для загрузки последней версии приложения. С новой версией при входе в аккаунт с учетной записью Google «Удалить объявление» все равно будет восстановлено.', 'нет', 'да']
['Недавно мы выпустили версию совершенно нового приложения, эта версия улучшила производительность и интегрировала все существующие языки, пожалуйста, посетите http://mangaworldapp.c

UnboundLocalError: local variable 'docs' referenced before assignment